In [52]:
# library for accessing API's based on url
# Import all python modules
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [143]:
# Cal-Adapt API
api = "https://api.cal-adapt.org/api"
headers = {'ContentType': 'json'}

# Point of interest - Glenn City Elementary School
point = 'POINT(-119.08096980316024 34.34871777505949)'

# Name of boundary layer in API (counties track)
resource = "counties"

# Request url
url = '%s/%s/' % (api,resource)

# Request params to find intersecting polygon
params = {'intersects': point, 'srs':4326, 'simplify':0.0001,'precision':4}

# Get geometry
response = requests.get(url, params=params, headers=headers)
if response.ok:
    params = {'intersects': point, 'srs':4326, 'simplify':.0001, 'precision':4}

# Get geometry
response = requests.get(url, params = params, headers=headers)
if response.ok:
    data = response.json()
    feature = data['features'][0]
    if feature:
        ref = '/api/%s/%s/' % (resource, feature['id'])
        params = {'ref': ref, 'stat': 'mean'}
        print('ref:', ref)
        location = 'Counties name %s' % feature['properties']['name']
        print('location:', location)
    else:
        print('Did not find any polygons that intersect your point')

ref: /api/counties/56/
location: Counties name Ventura


In [144]:
# Base url for extreme heat and %s is the placeholder for the slug
baseurl = 'https://api.cal-adapt.org/api/series/%s/events/'

def fetch_exheat(*urlargs, **params):
    # all possible combinations of the URL
    items = itertools.product(*urlargs)
    series = []
    for parts in items:
        slug = '_'.join(parts)
        print('Processing:', slug)
        # Sends GET request to the API using the URL constructed
        response = requests.get(baseurl % slug, params)
        data = response.json()
        # extracts the counts and converts to pandas series
        counts = pd.Series(data['data']).sort_index()
        # convert to date time remove time zones
        counts.index = pd.to_datetime(counts.index).tz_localize(None)
        # assign name of the series to the series object
        counts.name = slug
        # append series to list
        series.append(counts)
        # return panda dataframe
    return pd.concat(series, axis=1)

In [145]:
# Requesting object
climvar = ['tasmax']
period = ['day']
models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
scenarios = ['rcp45','rcp85']
#hmodelcounts = fetch_exheat(climvar, period, models, ['historical'], **params)
# Historical fata fetch
#hmodelcounts.columns = hmodelcounts.columns.str.replace('tasmax_day_', '')
projcounts = fetch_exheat(climvar, period, models, scenarios, **params)
# Projected data fetch
projcounts.columns = projcounts.columns.str.replace('tasmax_day_', '')
projcounts = projcounts.append(hmodelcounts).sort_index()
projcounts.head()


Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_203833/142259574.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  projcounts = projcounts.append(hmodelcounts).sort_index()


,CanESM2_rcp45,CanESM2_rcp85,CNRM-CM5_rcp45,CNRM-CM5_rcp85,HadGEM2-ES_rcp45,HadGEM2-ES_rcp85,MIROC5_rcp45,MIROC5_rcp85,CanESM2_historical,CNRM-CM5_historical,HadGEM2-ES_historical,MIROC5_historical
1970-01-01 00:00:00.000000000,284.498779,285.881378,293.527771,294.001282,285.914062,285.395691,290.577179,289.794037,NaN,NaN,NaN,NaN
1970-01-01 00:00:00.000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,285.317200,289.069611,286.450531,291.513397
1970-01-01 00:00:00.000000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289.380127,288.922607,286.382385,290.304108
1970-01-01 00:00:00.000000001,282.276917,283.466858,291.430847,291.252472,280.146271,286.636017,290.862518,290.628143,NaN,NaN,NaN,NaN
1970-01-01 00:00:00.000000002,283.724945,284.097473,289.470459,289.803619,278.752899,285.111938,290.984070,291.227631,NaN,NaN,NaN,NaN


In [146]:
url = 'https://api.cal-adapt.org/api/series/tasmax_day_CanESM2_rcp85/events/'

# Make request
response = requests.get(url, params=params)

df = pd.read_json(response.text, typ='series', orient='split')

In [136]:
print(df.head)

<bound method NDFrame.head of 1950-01-01 00:00:00+00:00    285.317200
1950-01-02 00:00:00+00:00    289.380127
1950-01-03 00:00:00+00:00    291.267059
1950-01-04 00:00:00+00:00    293.566528
1950-01-05 00:00:00+00:00    290.471313
                                ...    
2005-12-27 00:00:00+00:00    290.459961
2005-12-28 00:00:00+00:00    291.360565
2005-12-29 00:00:00+00:00    289.391571
2005-12-30 00:00:00+00:00    287.578033
2005-12-31 00:00:00+00:00    286.561707
Name: tasmax_day_CanESM2_historical, Length: 20454, dtype: float64>


In [14]:
hmodelcounts

In [150]:
obsurl = 'https://api.cal-adapt.org/api/series/tasmax_day_HadGEM2-ES_historical/exheat/' 

In [224]:
baseurl = 'https://api.cal-adapt.org/api/series/%s/exheat/'
params.update(freq='M')

def fetch_exheat(*urlargs, **params):
    items = itertools.product(*urlargs)
    series = []
    for parts in items:
        slug = '_'.join(parts)
        print('Processing:', slug)
        response = requests.get(baseurl % slug, params)
        data = response.json()
        counts = pd.Series(data['counts']).sort_index()
        counts.index = pd.to_datetime(counts.index).tz_localize(None)
        counts.name = slug
        series.append(counts)
    return pd.concat(series, axis=1)

In [225]:
climvar = ['tasmax']
period = ['day']
models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
scenarios = ['rcp45','rcp85']
hmodelcounts = fetch_exheat(climvar, period, models, ['historical'], **params)

projcounts = fetch_exheat(climvar, period, models, scenarios, **params)
projcounts = projcounts.append(hmodelcounts).sort_index()
projcounts.head()


Processing: tasmax_day_CanESM2_historical
Processing: tasmax_day_CNRM-CM5_historical
Processing: tasmax_day_HadGEM2-ES_historical
Processing: tasmax_day_MIROC5_historical
Processing: tasmax_day_CanESM2_rcp45
Processing: tasmax_day_CanESM2_rcp85
Processing: tasmax_day_CNRM-CM5_rcp45
Processing: tasmax_day_CNRM-CM5_rcp85
Processing: tasmax_day_HadGEM2-ES_rcp45
Processing: tasmax_day_HadGEM2-ES_rcp85
Processing: tasmax_day_MIROC5_rcp45
Processing: tasmax_day_MIROC5_rcp85


/tmp/ipykernel_203833/583059455.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  projcounts = projcounts.append(hmodelcounts).sort_index()


,tasmax_day_CanESM2_rcp45,tasmax_day_CanESM2_rcp85,tasmax_day_CNRM-CM5_rcp45,tasmax_day_CNRM-CM5_rcp85,tasmax_day_HadGEM2-ES_rcp45,tasmax_day_HadGEM2-ES_rcp85,tasmax_day_MIROC5_rcp45,tasmax_day_MIROC5_rcp85,tasmax_day_CanESM2_historical,tasmax_day_CNRM-CM5_historical,tasmax_day_HadGEM2-ES_historical,tasmax_day_MIROC5_historical
1950-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,2.0
1950-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,NaN,0.0
1950-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
1950-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
1950-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0


In [210]:
params.update(freq='M')
response = requests.get(obsurl, params)
data = response.json()
hmcounts = pd.DataFrame.from_dict(data)
hmcounts.index = pd.to_datetime(hmcounts.index).tz_convert(None)
hmcounts = pd.DataFrame(hmcounts['counts'])
hmcounts.rename(columns = {'counts' : ''})

,A
1951-08-31,1
1951-09-30,0
1951-10-31,1
1951-11-30,0
1951-12-31,0
...,...
2005-04-30,0
2005-05-31,0
2005-06-30,0
2005-07-31,0


In [203]:
hmcounts

,counts
1951-08-31,1
1951-09-30,0
1951-10-31,1
1951-11-30,0
1951-12-31,0
...,...
2005-04-30,0
2005-05-31,0
2005-06-30,0
2005-07-31,0


,counts
1951-08-31,1
1951-09-30,0
1951-10-31,1
1951-11-30,0
1951-12-31,0
...,...
2005-04-30,0
2005-05-31,0
2005-06-30,0
2005-07-31,0


In [165]:
data

{'98p': 88.79493530273436,
 'counts': {'1951-08-31T00:00:00+00:00': 1,
  '1951-09-30T00:00:00+00:00': 0,
  '1951-10-31T00:00:00+00:00': 1,
  '1951-11-30T00:00:00+00:00': 0,
  '1951-12-31T00:00:00+00:00': 0,
  '1952-01-31T00:00:00+00:00': 0,
  '1952-02-29T00:00:00+00:00': 0,
  '1952-03-31T00:00:00+00:00': 0,
  '1952-04-30T00:00:00+00:00': 0,
  '1952-05-31T00:00:00+00:00': 0,
  '1952-06-30T00:00:00+00:00': 0,
  '1952-07-31T00:00:00+00:00': 3,
  '1952-08-31T00:00:00+00:00': 2,
  '1952-09-30T00:00:00+00:00': 4,
  '1952-10-31T00:00:00+00:00': 0,
  '1952-11-30T00:00:00+00:00': 0,
  '1952-12-31T00:00:00+00:00': 0,
  '1953-01-31T00:00:00+00:00': 0,
  '1953-02-28T00:00:00+00:00': 0,
  '1953-03-31T00:00:00+00:00': 0,
  '1953-04-30T00:00:00+00:00': 0,
  '1953-05-31T00:00:00+00:00': 0,
  '1953-06-30T00:00:00+00:00': 0,
  '1953-07-31T00:00:00+00:00': 0,
  '1953-08-31T00:00:00+00:00': 0,
  '1953-09-30T00:00:00+00:00': 8,
  '1953-10-31T00:00:00+00:00': 0,
  '1953-11-30T00:00:00+00:00': 0,
  '1953-12-

In [156]:
type(hmcounts)

pandas.core.series.Series